# Algorithms



In [1]:
# import libraries and csv file

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import time
from numpy import mean

data = pd.read_csv('Cleaned_Data.csv')

In [2]:
data.head(10)

,truck_type,value,distance,special
0,Reefer,2024.01,2585,False
1,Reefer,1931.33,2106,False
2,Van,1709.99,3280,False
3,Reefer,3406.64,2082,False
4,Reefer,1189.01,2464,False
5,Van,1468.45,2234,False
6,Van,1355.33,3141,False
7,Van,1649.94,1786,True
8,Flatbed,2037.55,3375,False
9,Reefer,1874.20,3595,False


# Partition Algorithm

Function 1 - Baseline

- Start by writing a very simple algorithm to partition the data. This will act as your baseline for comparing your next one(s). This simple algorithm does not need to satisfy every constraint well (or at all!) but should at least produce the correct number of partitions, and include some randomization.

Requirements:
1. The user can decide to use any number of partitions, from 1, up to the size of the dataset
itself
2. No row should ever be duplicated, modified, or left out. Each row should appear in
exactly one partition.
3. Each partition should have as close to the same number of rows as each other
4. Each partition should have as close to the same number of rows from each category of
shipment, R, V, and F
5. The sum of the “value” column for each partition should be approximately the same
6. The sum of the “value” column for each class (R, V, F) in each partition should be
approximately the same.
7. Partitions should be randomized, so that repeated partitioning tasks would not
necessarily produce the same result each time.

In [3]:
def partitionFunction1():
    
    # get user input
    partitions = input("Enter number of partitions you would like: ")
    random_state = input("Enter random state integer if you want to replicate other paritions: ")
    
    # re-define inputs as integers
    random_state = int(random_state)
    partitions = int(partitions)
    
    # copy data into another dataframe so function can be called multiple times without reloading page
    iterable_data_set = data.copy()
    
    # define partition size and remainder to track during loop
    partition_size = iterable_data_set.shape[0] / partitions
    remainder = abs(partition_size - round(partition_size, 0))
    
    # initialize dictionary to store partitioned datasets
    partitioned_datasets = {}
    
    # loop through dataset
    for g in range(partitions):

        # update running average to adjust parition sizes
        running_average = len(iterable_data_set) / (partitions - g)

        # store remaining dataset for last parition
        if g == (partitions - 1):
            partitioned_datasets[g] = iterable_data_set

        # sample n + 1 if partition size is behind running average
        elif (running_average > partition_size):
            partitioned_datasets[g] = iterable_data_set.sample(n=int(partition_size + 1), random_state=random_state)
            sample_index_list = partitioned_datasets[g].index.tolist()
            iterable_data_set.drop(sample_index_list, axis=0, inplace=True)

        # sample n if partition size is ahead of running average
        else:
            partitioned_datasets[g] = iterable_data_set.sample(n=int(partition_size), random_state=random_state)
            sample_index_list = partitioned_datasets[g].index.tolist()
            iterable_data_set.drop(sample_index_list, axis=0, inplace=True)
            
    # output result
    return partitioned_datasets
            


In [4]:
partitionFunction1()

Enter number of partitions you would like: 999
Enter random state integer if you want to replicate other paritions: 1


{0:     truck_type    value  distance  special
 507     Reefer  1448.44      1978     True,
 1:     truck_type    value  distance  special
 508     Reefer  1311.18       792    False
 609     Reefer  1797.11      3406    False,
 2:     truck_type   value  distance  special
 358     Reefer  1683.5      3144    False,
 3:     truck_type    value  distance  special
 359     Reefer  1225.74      1976    False,
 4:     truck_type    value  distance  special
 360     Reefer  1148.25      1438    False,
 5:     truck_type   value  distance  special
 751    Flatbed  972.57       380    False,
 6:     truck_type    value  distance  special
 793    Flatbed  1794.73      2712    False,
 7:     truck_type    value  distance  special
 664     Reefer  1696.08      2766    False,
 8:     truck_type    value  distance  special
 665    Flatbed  5895.29      3693     True,
 9:     truck_type    value  distance  special
 320    Flatbed  1496.67      2360    False,
 10:     truck_type    value  distance  

1. Describe how this algorithm works.

In [5]:

# Algorithm Process Steps:

#     1 - Load data

#     2 - Get user input for partition size and randomizer value

#     3 - Define partition size

#     4 - Loop through dataset extracting loads from main dataset into another dictionary for analysis

#         4a - Track remainder_value - ie if partition size is uneven, the remainder will grow/ shrink based on 
#              whether you round up or down when sampling 

#         4b - if remainder value < average parition size --> round up on sample size (ceiling)

#              4b-1 - sample from dataset based on remainder value

#         4c - if remainder value > average parition size --> round down on sample size (floor)

#              4c-1 - sample from dataset based on remainder value

#         4d - add samples to dictionary (maintaining index)

#         4e - drop sample from initial dataset

#         4f - on the last partition add the remaining inital dataset

#      5 - END


2. What do the results look like when partitioning the data set?

In [6]:

# The algorithm returns loads on n_paritions as defined

# Based on the success metrics this algorithm succeeds in producing:
#    ~ same number of loads in each partition
#    no duplicates
#    no loads modified
#    randomized paritions (and seeded)


# Function 2 - Improving the Algorithm

Now write another algorithm that you think will improve the results on at least one of the
constraints.

In [7]:

# This first algorithm did not care about truck_type counts or sum of values

# This second algorithm will produce more consistent value sums of each partition

# To do this I will add one load at a time (vs group sampling) so the sampled load value sum 
# can be filtered


In [8]:
def partitionFunction2():

    # get user input
    partitions = input("Enter number of partitions you would like: ")
    random_state = input("Enter random state integer if you want to replicate other paritions: ")

    # re-define inputs as integers
    random_state = int(random_state)
    partitions = int(partitions)
    
    # copy data into another dataframe so function can be called multiple times without reloading page
    iterable_data_set = data.copy()
    
    # define partition size and remainder to track during loop
    partition_size = iterable_data_set.shape[0] / partitions
    
    # define average load value to maintain same value sum in each partition
    avg_load_value = iterable_data_set['value'].sum() / len(iterable_data_set) 
    
    # initialize dictionary to store partitioned datasets
    partitioned_datasets = {}
    
    # define target for parition load value count
    partition_value_sum_target = iterable_data_set['value'].sum() / partitions
    
    # loop through dataset
    for g in range(partitions):
        
        # initialize dataframe for partition[g]
        partitioned_datasets[g] = pd.DataFrame(columns=iterable_data_set.columns)
        
        # define running average target
        running_average = len(iterable_data_set) / (partitions - g)
        
        # set running values to 0 before each partition
        partition_value_sum_target_per_load = 0
        current_partition_value_sum = 0
        
        # store remaining dataset for last parition
        if g == (partitions - 1):
            partitioned_datasets[g] = iterable_data_set
            
        # store partions when we do not need to increase to parition size
        elif (running_average < partition_size):
            
            # store partions when we do not need to increase to parition size
            for i in range(math.floor(partition_size)):
                
                # get average value count for each parition load so we can adjust accordingly
                partition_value_sum_target_per_load = partition_value_sum_target_per_load + (partition_value_sum_target / math.floor(partition_size))
                
                # check if the partition value sum is lagging behind running average
                if (i != 0 and current_partition_value_sum < partition_value_sum_target_per_load):
                    
                    # if so, resample so load value is larger than the average
                    load = iterable_data_set[iterable_data_set['value'] > mean(iterable_data_set['value'])].sample(random_state=random_state)

                # check if the partition value sum is ahead of running average
                elif (i != 0 and current_partition_value_sum > partition_value_sum_target_per_load): 

                    # if so, resample so load value is smaller than the average
                    load = iterable_data_set[iterable_data_set['value'] < mean(iterable_data_set['value'])].sample(random_state=random_state)
                
                # sample a load when i = 0
                else:
                    load = iterable_data_set.sample(random_state=random_state)

                # add sampled load to partition
                partitioned_datasets[g] = partitioned_datasets[g].append(load.iloc[[0]])

                # drop sampled load from initial dataset
                iterable_data_set.drop(load.index.tolist(), axis=0, inplace=True)
                
                # get current current parition value sum
                current_partition_value_sum = partitioned_datasets[g]['value'].sum()
                
        # store partions when we do need to increase to parition size
        else:
            
            # loop through initial dataset to define parition[g]
            for i in range(math.ceil(partition_size)):
                
                # get average value count for each parition load so we can adjust accordingly
                partition_value_sum_target_per_load = partition_value_sum_target_per_load + (partition_value_sum_target / math.floor(partition_size))
                
                # check if the partition value sum is lagging behind running average
                if (i != 0 and current_partition_value_sum < partition_value_sum_target_per_load): 

                    # if so, resample so load value is larger than the average
                    load = iterable_data_set[iterable_data_set['value'] > mean(iterable_data_set['value'])].sample(random_state=random_state)

                # check if the partition value sum is ahead of running average
                elif (i != 0 and current_partition_value_sum > partition_value_sum_target_per_load):

                    # if so, resample so load value is smaller than the average
                    load = iterable_data_set[iterable_data_set['value'] < mean(iterable_data_set['value'])].sample(random_state=random_state)
                    
                # sample a load when i = 0
                else:
                    load = iterable_data_set.sample(random_state=random_state)

                # add sampled load to partition
                partitioned_datasets[g] = partitioned_datasets[g].append(load.iloc[[0]])

                # drop sampled load from initial dataset
                iterable_data_set.drop(load.index.tolist(), axis=0, inplace=True)
                
                # get current current parition value sum
                current_partition_value_sum = partitioned_datasets[g]['value'].sum()

    # output result
    return partitioned_datasets


# Compare Algorithms

- create a function to compare algorithms

In [9]:
# this function can report the output of the algorithm (algorithm_output) or summary metrics

def AlgorithmResults(function):
    
    # run algorithm
    partitioned_datasets = function()
    
    # initilize a dataframe for test function results to be displayed
    columns = ['Load Count', 
               'Duplicates', 
               'Sum of Values'
              ]
    index = range(0, len(partitioned_datasets))
    algorithm_output = pd.DataFrame(columns=columns, index=index)
    
    # initialize list to store index arrays for duplication check
    index_list = []

    # loop thru partitions to extract index lists
    for i in range(len(partitioned_datasets)):
        
        # get index list of each partition
        algorithm_output.iloc[i, 0] = len(partitioned_datasets[i])
        
        # get duplicates in each partition
        duplicates = [number for number in (partitioned_datasets[i].index.tolist()) if (partitioned_datasets[i].index.tolist()).count(number) > 1]
        algorithm_output.iloc[i, 1] = len(duplicates)
        
        # get sum of partition value column
        algorithm_output.iloc[i, 2] = partitioned_datasets[i]['value'].sum()
                
        # add index's to list
        index_list += partitioned_datasets[i].index.tolist()
    
    # get duplicates of for all partitions and raise error if there are duplicates
    duplicates_all = [number for number in index_list if index_list.count(number) > 1]
    if sum(duplicates_all) > 1:
        raise Exception("There are duplicates in some partitions!")
        
    # define metrics summary table
    metric_summary = pd.DataFrame(index=range(0,1), columns =  ['Load Count',
                                                                 'Duplicate Count',
                                                                 'Value Sum StDev'])

    # populate metrics summary
    metric_summary['Load Count'][0] = algorithm_output['Load Count'].sum()
    metric_summary['Duplicate Count'][0] = round(algorithm_output['Duplicates'].sum(), 2)
    metric_summary['Value Sum StDev'][0] = round(algorithm_output['Sum of Values'].std(), 0)
    
    return metric_summary
    

# Algorithm Comparison

Compare partition = 5:

In [10]:
AlgorithmResults(partitionFunction1)

Enter number of partitions you would like: 5
Enter random state integer if you want to replicate other paritions: 2


,Load Count,Duplicate Count,Value Sum StDev
0,1000,0,18184.0


In [11]:
AlgorithmResults(partitionFunction2)

Enter number of partitions you would like: 5
Enter random state integer if you want to replicate other paritions: 2


,Load Count,Duplicate Count,Value Sum StDev
0,1000,0,7363.0


Compare partition = 10:

In [12]:
AlgorithmResults(partitionFunction1)

Enter number of partitions you would like: 10
Enter random state integer if you want to replicate other paritions: 21


,Load Count,Duplicate Count,Value Sum StDev
0,1000,0,12672.0


In [13]:
AlgorithmResults(partitionFunction2)

Enter number of partitions you would like: 10
Enter random state integer if you want to replicate other paritions: 21


,Load Count,Duplicate Count,Value Sum StDev
0,1000,0,8037.0


Compare partition = 100:

In [14]:
AlgorithmResults(partitionFunction1)

Enter number of partitions you would like: 100
Enter random state integer if you want to replicate other paritions: 45


,Load Count,Duplicate Count,Value Sum StDev
0,1000,0,4843.0


In [15]:
AlgorithmResults(partitionFunction2)

Enter number of partitions you would like: 100
Enter random state integer if you want to replicate other paritions: 45


,Load Count,Duplicate Count,Value Sum StDev
0,1000,0,4659.0


# Function 2 Questions

1. Describe how this algorithm works.

In [16]:

# Algorithm Process Steps:

#     1 - Load data

#     2 - Get user input for partition size and randomizer value

#     3 - Define partition size

#     4 - Loop through dataset extracting loads from main dataset into another dictionary for analysis

#         4a - Track remainder_value - ie if partition size is uneven, the remainder will grow/ shrink based on 
#              whether you round up or down when sampling 

#         4b - if remainder value < average parition size --> round up on sample size (ceiling)

#              4b-1 - if current partition value sum < running average --> resample for higher value
#              4b-2 - if current partition value sum > running average --> resample for lower value

#         4c - if remainder value > average parition size --> round down on sample size (floor)

#              4c-1 - if current partition value sum < running average --> resample for higher value
#              4c-2 - if current partition value sum > running average --> resample for lower value

#         4d - add sample to dictionary (maintaining index)

#         4e - drop sample from initial dataset

#         4f - on the last partition add the remaining inital dataset

#      5 - END


2. Did the algorithm improve the results compared to your baseline case? How do you know?

In [17]:

# based on the test function the second algorithm improved the value sum of each partition
# see above algorithm comparisons

# if a large outlier is chosen for the first partition load, the effectiveness of the algorithm
# can be reduced

# the benefit seen decreases as partition sizes increases due to the partition value sum
# decreasing as a %


3. What can still be improved with this algorithm?

In [18]:

# make truck type counts and truck type value counts between partitions more equal


# Section 3 - Final Questions

1. What other ideas do you have for how to implement this algorithm? Describe how they might work, and why they might help improve the results.

In [19]:

# choosing the intiial load close to the mean of the dataset (vs random) would help but 
# this would reduce the random-ness of the algorithm

# the same approach in algorithm 2 can be used to dial in truck type counts and their 
# respective value sums

# example of optimizing truck type count:
# when adding each load to parition -->
#    track current_partition_truck_type_count vs running truck type count average --> 
#        then adjust sampling choice by which truck type counts are less and add them

# with more time I would simplify the code (implement more functions vs procedural 
# programming) so maininting this in a production setting would be easier


2. When might you use an algorithm like these ones in a business setting?

In [20]:

# if you want to see which combination of loads produce the least or most 
# value/truck_type/special_type 

# ie we do not want any special loads but also want to maintain the same value sum, which 
# combination of loads produces this?


3. Under what circumstances, if any, would the baseline algorithm likely be sufficient to satisfy all the constraints?

In [21]:

# defining what "close to the same number" tolerance is for the metrics
# if the tolerance is large enough, the first algorithm may work

# if randomizing was not a requirement, the algorithm could be optimized for all metrics
#     ie sort initial dataset and choose values based on running averages

# the 'value' column outliers make randomizing while trying to keep the running averages the 
# same difficult
